In [7]:
!pip install fastapi
!pip install uvicorn

  Using cached fastapi-0.116.1-py3-none-any.whl.metadata (28 kB)
  Using cached starlette-0.47.3-py3-none-any.whl.metadata (6.2 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
Using cached fastapi-0.116.1-py3-none-any.whl (95 kB)
Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
Using cached pydantic_core-2.33.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
Using cached starlette-0.47.3-py3-none-any.whl (72 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached typing_inspection-0.4.1-py3-none-any.whl (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [fastapi]m4/6 [starlette]
  Using cached uvicorn-0.35.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached cli

In [9]:
!pip install transformers
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 17.8 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 19.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers] [transformers]


In [11]:
!pip install chromadb

  Using cached chromadb-1.0.20-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.3 kB)
  Using cached build-1.3.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached opentelemetry_api-1.36.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.36.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_sdk-1.36.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached bcrypt-4.3.0-cp39-abi3-manylinux_2_34_x86_64.whl.metadata (10 kB)
  Using cached kubernetes-33.1.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached orjson-3.11.3-cp310-cp310-manylinu

In [12]:
import os
from fastapi import FastAPI, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
from starlette.responses import Response
import PIL.Image as Image

import numpy as np
#import cv2
import io

from transformers import ViTModel, ViTImageProcessorFast

import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from chromadb.utils.data_loaders import ImageLoader

import json

In [13]:
 def receive_image(img):
    contents = img.file.read()

    working_image = Image.open(io.BytesIO(contents))

    directory = './api/working_directory/working_image.jpg'

    working_image.save(directory)

    #instantiate the image loader that ChromaDB uses to load pictures

    image_loader = ImageLoader()

    #intantiate the custom embedding function

    image_embbeding_function = GoogleVITHuge224Embedding()

    #connect to the database

    chroma_client = chromadb.PersistentClient(path='./models/google_vit_sample1000_db')

    #connect to the correct collection

    images_db = chroma_client.get_or_create_collection(name="google_vit_sample1000_collection", embedding_function=image_embbeding_function, data_loader=image_loader)

    #test picture string

    query_uris = '../raw_data/test_images/Two_Young_Girls_at_the_Piano_MET_rl1975.1.201.R.jpg'
    #query_uris = '/Users/shogun/code/gwen-m97/raw_data/test_images/Piet_Mondriaan,_1942_-_New_York_City_I.jpg'
    #query_uris = '/Users/shogun/code/gwen-m97/raw_data/test_images/Paul_Cézanne_-_The_Basket_of_Apples_-_1926.252_-_Art_Institute_of_Chicago.jpg'
    #query_uris = '/Users/shogun/code/gwen-m97/raw_data/test_images/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg'
    #query_uris = './api/working_directory/working_image.jpg'

    #perform the query

    image_suggestions = images_db.query(
    query_uris=[query_uris], include=['uris','metadatas'], n_results=5
)

    #url=f"{image_suggestions['metadatas'][0][9]['img']}"
    image_dict = {'image_1': image_suggestions['metadatas'][0][0]['img'],
                  'image_2': image_suggestions['metadatas'][0][1]['img'],
                  'image_3': image_suggestions['metadatas'][0][2]['img'],
                  'image_4': image_suggestions['metadatas'][0][3]['img'],
                  'image_5': image_suggestions['metadatas'][0][4]['img']
                  }

    return_json = json.dumps(image_dict, indent=4)

    #annotated_img = cv2_img

    ### Encoding and responding with the image
    #im = cv2.imencode('.jpg', image)[1] # extension depends on which format is sent from Streamlit
    #return image_suggestions
    #return Response(image_suggestions['metadatas'][0]['img'])

    os.remove(query_uris)

    return return_json

In [19]:
from fastapi import UploadFile
import io

path = "../raw_data/Images to try/Paul_Cézanne_-_The_Basket_of_Apples_-_1926.252_-_Art_Institute_of_Chicago.jpg"

with open(path, "rb") as f:
    file_bytes = f.read()

# Simuler un UploadFile
fake_upload_file = UploadFile(filename="Paul_Cézanne.jpg", file=io.BytesIO(file_bytes))

# Appeler ta fonction
result_json = receive_image(fake_upload_file)
print(result_json)


FileNotFoundError: [Errno 2] No such file or directory: '../raw_data/Images to try/Paul_Cézanne_-_The_Basket_of_Apples_-_1926.252_-_Art_Institute_of_Chicago.jpg'